# Exercises of difficulty 5 in R

In [2]:
library(testthat)

# 1. Some Egyptian fractions

Given a rational number n- n >= 0, with denominator strictly positive decompose 
this number as a sum of rationals with numerators equal to one and without 
repetitions (2/3 = 1/2 + 1/6 is correct but not 2/3 = 1/3 + 1/3, 1/3 is repeated).

The algorithm must be "greedy", so at each stage the new rational obtained in the 
decomposition must have a denominator as small as possible. In this manner the 
sum of a few fractions in the decomposition gives a rather good approximation of
the rational to decompose.

2/3 = 1/3 + 1/3 doesn't fit because of the repetition but also because the first 
1/3 has a denominator bigger than the one in 1/2 in the decomposition 2/3 = 1/2 + 1/6.

### Example

    decompose("21/23") --> "1/2,1/3,1/13,1/359,1/644046"
    
    
#### Note

1) The decomposition of 21/23 as

    21/23 = 1/2 + 1/3 + 1/13 + 1/598 + 1/897

is exact but don't fulfill our requirement because 598 is bigger than 359. Same for

    21/23 = 1/2 + 1/3 + 1/23 + 1/46 + 1/69 (23 is bigger than 13)
or 

    21/23 = 1/2 + 1/3 + 1/15 + 1/110 + 1/253 (15 is bigger than 13).

2) The rational given to decompose could be greater than one or equal to one, in
which case the first "fraction" will be an integer (with an implicit denominator of 1). 

In [3]:
decompose <- function(n) {
  options(digits=22)
  
  # Return 'n' if it is a integer
  if (n %% 1 == 0 & n > 1) return(as.character(n))
  
  # Define vector to store results & set counter
  decomposition <- c(); return_str <- c(); i <- 2
  
  while(sum(decomposition) < n) {
    if ((n - (sum(decomposition) + 1/i)) >= 0) {
      decomposition <- c(decomposition, 1/i)
      return_str    <- c(return_str, paste0("1/", i))
    }
  i <- i + 1
  }
  return(paste(return_str, collapse = ","))
}

In [4]:
dotest <- function(n, expected) {
  actual <- decompose(n)
  expect_equal(actual, expected)
}

test_that("tests", {
  dotest(3/4, "1/2,1/4")
  dotest(12/4, "3")
})

# 2. Where my anagrams at?

What is an anagram? Well, two words are anagrams of each other if they both contain
the same letters.
For example:

    'abba' & 'baab' == true
    'abba' & 'baab' == true
    'abba' & 'abbba' == false
    'abba' & 'abca' == false
    
Write a function that will find all the anagrams of a word from a list. You will
be given two inputs a word and an array with words. You should return an array of
all the anagrams or an empty array if there are none. 
For example:

    anagrams('abba', ['aabb', 'abcd', 'bbaa', 'dada']) => ['aabb', 'bbaa']
    anagrams('racer', ['crazer', 'carer', 'racar', 'caers', 'racer']) => ['carer', 'racer']
    anagrams('laser', ['lazing', 'lazy',  'lacer']) => []

In [5]:
anagrams <- function(word, words) {
  
  # Get single letters & their freq of 'word' adn each element in 'words'
  word_letters <- table(strsplit(word, split = ""))
  words_letters <- lapply(words, function(x) table(strsplit(x, split = "")))
  
  # Compare letters & their freq of each element in 'words_letters' to 'word_letters'
  anagram_idx <- sapply(words_letters, function(x) isTRUE(all.equal(word_letters, x)))

  # Return elements of 'words' that are a anagram of 'word'
  words[anagram_idx]
}

In [6]:
test_that("Sample Tests", {
  expect_equal(anagrams('abba', c('aabb', 'abcd', 'bbaa', 'dada')), c('aabb', 'bbaa'))
  expect_equal(anagrams('racer', c('crazer', 'carer', 'racar', 'caers', 'racer')), c('carer', 'racer'))
})

# 3. Primes in numbers

Given a positive number n > 1 find the prime factor decomposition of n. 
The result will be a string with the following form :
    
    "(p1**n1)(p2**n2)...(pk**nk)"
    
    where 'a ** b' means a to the power of b

with the p(i) in increasing order and n(i) empty if n(i) is 1.

Example: 
    
    n = 86240 should return "(2**5)(5)(7**2)(11)"

In [8]:
primeFactors <- function(n) {
  # Define index 'i' & vector 'res' to store the primes of the decomposition
  res <- c(); i <- 2
  
  # Do the factor decomposition for 'n'. Divide it by 'i' & if the result in a
  # round numeric, update n to n/i - repeat this producere until n < i
  while(n >= i) {
    if ((n / i) %% 1 == 0) {
      n = n / i
      res = c(res, i)
    } else {
      i = i + 1
    }
  }
  
  # If 'res' is null, 'n' is a prime -> no decomposition possible -> return 'n'
  # Else use 'res' to create the decomposition as a single string
  if (is.null(res)) {
    
    return(paste0('(', n, ')'))
  } else {
    
    tab_res <- table(res)
    
    ret_res <- paste(unlist(
      lapply(seq_along(tab_res), FUN = function(x) {
        paste0('(', names(tab_res[x]), "**" , tab_res[x], ')')
      }
      )), collapse = '')
    
    return(gsub('\\*\\*1)', ')',  ret_res))
  }
}

In [9]:
testing <- function(n, expected) {
  actual <- primeFactors(n)
  expect_equal(actual, expected)
}

test_that("tests", {
  testing(7775460, "(2**2)(3**3)(5)(7)(11**2)(17)")
  testing(7919, "(7919)")
})

# 4. Last digit of a large number

Define a function that takes in two non-negative integers 'aaa' & 'bbb' and  
returns the last decimal digit of aba^bab.
Note that 'aaa' and 'bbb' may be very large!

### For example:
The last decimal digit of 9^7 is 0, as 9^7 = 4782969
The last decimal digit of (2^200)^(2^300), which has over 10^{92} decimal digits,
is 666.
Also, please take 0^0 to be 1.

You may assume that the input will always be valid.

### Examples:
    last_digit("4", "1")            # returns 4
    last_digit("4", "2")            # returns 6
    last_digit("9", "7")            # returns 9    
    last_digit("10","10000000000")  # returns 0
    
## Remark
Since R doesn't have native arbitrarily large integers, your arguments are going
to be strings representing non-negative integers instead.

In [10]:
last_digit <- function(s1, s2) {
  if (s2 == 0) return(1)
  if (s1 == 0) return(0)

  s1 <- as.numeric(substr(s1, nchar(s1), nchar(s1)))
  s2 <- as.numeric(substr(s2, nchar(s2)-1, nchar(s2))) 
 
  if (s1 %in% c(0,1,5,6)) return(s1)
  if (s1 == 4) return(c(6,4)[s2 %% 2 +1])
  if (s1 == 9) return(c(1,9)[s2 %% 2 +1])

  if (s1 == 2) return(c(6,2,4,8)[s2 %% 4 +1])
  if (s1 == 3) return(c(1,3,9,7)[s2 %% 4 +1])
  if (s1 == 7) return(c(1,7,9,3)[s2 %% 4 +1])
  if (s1 == 8) return(c(6,8,4,2)[s2 %% 4 +1])
}

In [11]:
expect_equal(last_digit("4", "1"), 4)
expect_equal(last_digit("4", "2"), 6)
expect_equal(last_digit("9", "7"), 9)
expect_equal(last_digit("10", "10000000000"), 0)
expect_equal(last_digit("1606938044258990275541962092341162602522202993782792835301376", "2037035976334486086268445688409378161051468393665936250636140449354381299763336706183397376"), 6)
expect_equal(last_digit("3715290469715693021198967285016729344580685479654510946723", "68819615221552997273737174557165657483427362207517952651"), 7)

# 5. Product of consecutive Fib numbers
The Fibonacci numbers are the numbers in the following integer sequence (Fn):
    
    0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, ...

whereby

    F(n) = F(n-1) + F(n-2) with F(0) = 0 and F(1) = 1.

Given a number 'prod' we search two Fibonacci numbers F(n) and F(n+1) verifying

    F(n) * F(n+1) = prod

Your function productFib() takes an integer 'prod' and returns an array depending whether 
there exists two Fibonacci numbers, which product equals 'prod'
If it exists, return: 

    c(F(n), F(n+1), True)

If not, return:

    c(F(n), F(n+1), False)
    
### Examples
    > productFib(714) # should return (21, 34, true), 
                      # since F(8) = 21, F(9) = 34 and 714 = 21 * 34

    > productFib(800) # should return (34, 55, false), 
                      # since F(8) = 21, F(9) = 34, F(10) = 55 and 21 * 34 < 800 < 34 * 55
                    
    > productFib(714) # should return [21, 34, true], 
    > productFib(800) # should return [34, 55, false], 
    > productFib(714) # should return {21, 34, 1}, 
    > productFib(800) # should return {34, 55, 0},        
    > productFib(714) # should return {21, 34, true}, 
    > productFib(800) # should return {34, 55, false}, 

In [73]:
fib <- function(n) {
    if(n <= 1) {
        return(n)
    } else {
    return(fib(n-1) + fib(n-2))
    }
}

productFib <- function(prod) {
    fib1 = 0
    fib2 = 1
    while(fib1 * fib2 < prod) {
        fib2_old = fib2
        fib2     = fib1 + fib2
        fib1     = fib2_old   
    }
    if (fib1 * fib2 == prod) {
        return(c(fib1, fib2, TRUE))
    } else {
        return(c(fib1, fib2, FALSE))
    }
}

In [75]:
dotest <- function(prod, expected) {
    actual <- productFib(prod)
    expect_equal(actual, expected)
}

test_that("tests productFib", {
    dotest(4895, c(55, 89, TRUE))
    dotest(5895, c(89, 144, FALSE))
})

# 6. Buddy Pairs
You know what divisors of a number are. The divisors of a positive integer n are said to be proper when you consider only the divisors other than n itself. In the following description, divisors will mean proper divisors. For example for 100 they are 1, 2, 4, 5, 10, 20, 25, and 50.

Let s(n) be the sum of these proper divisors of n. Call buddy two positive integers such that the sum of the proper divisors of each number is one more than the other number:

(n, m) are a pair of buddy if s(m) = n + 1 and s(n) = m + 1

For example 48 & 75 is such a pair:

    Divisors of 48 are: 1, 2, 3, 4, 6, 8, 12, 16, 24 --> sum: 76 = 75 + 1
    Divisors of 75 are: 1, 3, 5, 15, 25 --> sum: 49 = 48 + 1

### Task

Given two positive integers start and limit, the function buddy(start, limit) should return the first pair (n m) of buddy pairs such that n (positive integer) is between start (inclusive) and limit (inclusive); m can be greater than limit and has to be greater than n

If there is no buddy pair satisfying the conditions, then return "Nothing" or (for Go lang) nil or (for Dart) null; (for Lua, Pascal, Perl) [-1, -1].
Examples

(depending on the languages)

    buddy(10, 50) returns [48, 75] 
    buddy(48, 50) returns [48, 75]
    or
    buddy(10, 50) returns "(48 75)"
    buddy(48, 50) returns "(48 75)"


In [33]:
# Helpfunction to get the sum of the possible dividors of 'n'
s <- function(n) {
    
    # Get the primes of 'n'
    primes <- which(!n %% (1:sqrt(n)))
    
    # Get all possible dividors and sum them up
    sum(unique(c(primes, rev((n / primes)[-1]))))
}

buddy <- function(start, limit) {
    
    # Check whether we can find a buddy pair with 'n' in [start; limit]
    for (n in start:limit) {
        m <- s(n) - 1
        if (n == s(m) - 1 && m > n) return(sprintf('(%d %d)', n, m))
    }
    'Nothing'
}

In [34]:
testing <- function(st, lim, expect) {
  actual <- buddy(st, lim)
  cat("start ", st, "limit ", lim, "\n")
  cat("actual" , actual, "\n")
  cat("expect" , expect, "\n\n")
  expect_equal(actual, expect)
}

test_that("decomp", {
  testing(654, 3567, "(1050 1925)");
  testing(23, 4669, "(48 75)");
  testing(6379, 8275, "Nothing");
})

start  654 limit  3567 
actual (1050 1925) 
expect (1050 1925) 

start  23 limit  4669 
actual (48 75) 
expect (48 75) 

start  6379 limit  8275 
actual Nothing 
expect Nothing 



# 7. Perimeter of squares in a rectangle
The drawing shows 6 squares the sides of which have a length of 1, 1, 2, 3, 5, 8. It's easy to see that the sum of the perimeters of these squares is : 4 * (1 + 1 + 2 + 3 + 5 + 8) = 4 * 20 = 80

Could you give the sum of the perimeters of all the squares in a rectangle when there are n + 1 squares disposed in the same manner as in the drawing 
(s. https://www.codewars.com/kata/559a28007caad2ac4e000083/train/r)

    perimeter(5)  should return 80
    perimeter(7)  should return 216
The function perimeter has for parameter n where n + 1 is the number of squares (they are numbered from 0 to n) and returns the total perimeter of all the squares.


In [39]:
# Get the Fibonacci-sequence from 0 - n
fib_seq <- function(n) {
    
    # Check edge-cases with n in [0;1]
    if (n == 0) return(c(1))
    if (n == 1) return(c(1, 1))
    
    # Initalize fib_seq
    fib_seq <- c(1,1)
    
    # Get the values for the indeces 3:n (sum of the previous two entrances)
    for(i in 3:n) fib_seq[i] = fib_seq[i-1] + fib_seq[i-2]
    
    # Return the sequence
    return(fib_seq)
}
perimeter <- function (n) {
    
    # Create Fibonacci-Seq. from 1 to n+1
    fib_seq <- fib_seq(n + 1)
                      
    # return 4 * sum(fib_seq)
    return(4 * sum(fib_seq))
}

In [40]:
dotest <- function(n, expected) {
    actual <- perimeter(n)
    expect_equal(actual, expected)
}

test_that("tests perimeter", {
    dotest(5, 80)
    dotest(7, 216)
    dotest(20, 114624)
    dotest(30, 14098308)
})

# 8 Is my friend cheating?

- A friend of mine takes the sequence of all numbers from 1 to n (where n > 0).
- Within that sequence, he chooses two numbers, a and b.
- He says that the product of a and b should be equal to the sum of all numbers in the sequence, excluding a and b.
- Given a number n, could you tell me the numbers he excluded from the sequence?

The function takes the parameter: n (n is always strictly greater than 0) and returns an array or a string (depending on the language) of the form: 

    > [(a, b), ...] or [[a, b], ...] or {{a, b}, ...} or or [{a, b}, ...]
    > with all (a, b) which are the possible removed numbers in the sequence 1 to n
    > [(a, b), ...] or [[a, b], ...] or {{a, b}, ...} or ... will be sorted in increasing order of the "a"
    
It happens that there are several possible (a, b). The function returns an empty array (or an empty string) if no possible numbers are found which will prove that my friend has not told the truth! 

### Examples:

    removNb(26) should return list(c(15, 21), c(21, 15))

In [85]:
removeNb <- function(n) {
    # Calc the sum of the seq. 1-n & create a list to store solutions
    sum_seq  = sum(seq_len(n))
    solution = list()

    for (a in seq_len(n)) {

        # Check if there is a element X in the seq, such that 'a' times the element 
        # 'X' is equal to: sum(1:n) - ('a' + X). If so, add these elements to 'solution'
        check = which(a %*% seq_len(n) == sum(seq_len(n)) - (a + seq_len(n)))
        
        if (length(check) > 0) solution[[length(solution) + 1]] = c(a, check)
    }
    solution
}

In [86]:
testing <- function(n, expected) {
    actual <- removeNb(n)
    expect_equal(actual, expected)
}

test_that("tests removeNb", {
    testing(26, list(c(15, 21), c(21, 15)))
    testing(100, list())
    testing(1234, list(c(787, 966), c(966, 787)))
})